# 数据集对象使用方法

[![](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/docs/mindspore/programming_guide/source_zh_cn/dataset_usage.ipynb)&emsp;[![](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/programming_guide/zh_cn/mindspore_dataset_usage.ipynb)

## 概述

原始数据集通过数据集加载接口读取上来，并进行数据增强操作产生的数据集对象（dataset），有两种常规的使用方法如下所示：

- 通过创建迭代器来获取数据。

- 传入Model接口（如model.train、model.eval等）进行训练或推理。

## 通过创建迭代器来获取数据

- 针对创建好的dataset对象，通常可以创建两类迭代器来遍历dataset对象以便获取数据，具体为元组迭代器和字典迭代器。

- 创建元组迭代器对应的接口为create_tuple_iterator，创建字典迭代器对应的接口为create_dict_iterator，具体使用方法如下。

In [1]:
import mindspore.dataset as ds

np_data = [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]
dataset = ds.NumpySlicesDataset(np_data, column_names=["col1"], shuffle=False)

# 创建tuple类型迭代器
print("\n create tuple iterator to iterate dataset")
for item in dataset.create_tuple_iterator():
    print("item[0]:\n", item[0])

# 创建dict类型的迭代器
print("\n create dict iterator to iterate dataset")
for item in dataset.create_dict_iterator():
    print("item[col1]:\n", item["col1"])

# 直接遍历dataset对象（等同于创建tuple iterator）
print("\n iterate dataset object directly")
for item in dataset:
    print("item[0]:\n", item[0])

# 使用enumerate方式遍历（等同于创建tuple iterator）
print("\n using enumerate way to iterate dataset object")
for index, item in enumerate(dataset):
    print("index: {}, item[0]:\n {}".format(index, item[0]))

- 具体可参考[create_tuple_iterator](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/dataset/mindspore.dataset.NumpySlicesDataset.html#mindspore.dataset.NumpySlicesDataset.create_tuple_iterator) 和[create_dict_iterator](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/dataset/mindspore.dataset.NumpySlicesDataset.html#mindspore.dataset.NumpySlicesDataset.create_dict_iterator)的API参数说明。

- 如果需要产生多个epoch的数据，可通过num_epochs进行控制。

- 默认输出的mindspore.Tensor(即output_numpy=False)，如果希望输出是numpy array，需要设置参数output_numpy=True。

In [2]:
# 创建tuple类型迭代器，默认输出为mindspore.Tensor
# 通过传入num_epoch参数告知需产生几个epoch数据，然后可遍历创建出的迭代器num_epoch次，如下所示
# 相比每次调用一次create_tuple_iterator(num_epoch=1)， 调用两次，如下方式传入num_epochs=2，有助于提高pipeline性能

epoch = 2
iterator = dataset.create_tuple_iterator(num_epochs=epoch)
for _ in range(epoch):
    for item in iterator:
        print("\ntype of item[0]:", type(item[0]), "\nitem[0]:\n ", item) 


# 创建tuple类型迭代器，并使输出为numpy array
for item in dataset.create_tuple_iterator(output_numpy=True):
    print("\ntype of item[0]:", type(item[0]), "\nitem[0]:", item) 

## 传入Model接口进行训练或推理

- 通过`Model`接口，将`dataset`传入网络，喂入网络进行训练。

- `Model`接口初始化之后，可以将dataset传入model.train/model.eval接口进行训练/推理，具体参数可参考API说明。

- `dataset_sink_mode`参数说明：表示是否下沉到device，如果不下沉则相当于上述创建迭代器，逐条获取数据并喂入网络；如果为True则会将数据发送到设备上，并喂入到网络中。

In [3]:
import numpy as np
from mindspore import ms_function
from mindspore import context, nn, Model
import mindspore.dataset as ds
import mindspore.ops as ops


def create_dataset():
    np_data = [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]
    np_data = np.array(np_data, dtype=np.float16)
    dataset = ds.NumpySlicesDataset(np_data, column_names=["col1"], shuffle=False)
    return dataset


class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = ops.ReLU()
        self.print = ops.Print()

    @ms_function
    def construct(self, x):
        self.print(x)
        return self.relu(x)


if __name__ == "__main__":
    # it is supported to run in CPU, GPU or Ascend 
    context.set_context(mode=context.GRAPH_MODE)
    dataset = create_dataset()
    network = Net()
    model = Model(network)
    
    # do training, sink to device defaultly
    model.train(epoch=1, train_dataset=dataset, dataset_sink_mode=True)